# Titanic Survival Prediction using Machine Learning

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd

### Load the dataset

In [ ]:
titanic_data, info = tfds.load('titanic', split='train', as_supervised=False, with_info=True)
titanic_df = tfds.as_dataframe(titanic_data, info)
train_data, test_data = train_test_split(titanic_df, test_size=0.2, random_state=42)

### Preprocess the dataset

In [4]:
titanic_df.head()

,age,boat,body,cabin,embarked,fare,home.dest,name,parch,pclass,sex,sibsp,survived,ticket
0,30.0,b'Unknown',-1,b'Unknown',2,13.0000,"b'Sarnia, ON'","b'McCrie, Mr. James Matthew'",0,1,0,0,0,b'233478'
1,37.0,b'Unknown',98,b'Unknown',2,7.9250,"b'Ruotsinphytaa, Finland New York, NY'","b'Gustafsson, Mr. Anders Vilhelm'",0,2,0,2,0,b'3101276'
2,28.0,b'9',-1,b'Unknown',2,13.0000,b'Spain',"b'Reynaldo, Ms. Encarnacion'",0,1,1,0,1,b'230434'
3,18.0,b'Unknown',-1,b'Unknown',2,73.5000,"b'Lyndhurst, England'","b'Davies, Mr. Charles Henry'",0,1,0,0,0,b'S.O.C. 14879'
4,-1.0,b'Unknown',-1,b'Unknown',0,7.8958,b'Unknown',"b'Gheorgheff, Mr. Stanio'",0,2,0,0,0,b'349254'


In [5]:
# 1. Encode categorical features
encoder = preprocessing.OneHotEncoder(sparse_output=False)
train_data['sex'] = encoder.fit_transform(train_data['sex'].values.reshape(-1, 1))
test_data['sex'] = encoder.transform(test_data['sex'].values.reshape(-1, 1))

In [6]:
# 2. Handle missing values
train_data['age'].fillna(train_data['age'].median(), inplace=True)
test_data['age'].fillna(train_data['age'].median(), inplace=True)

In [7]:
# 3.Scale numerical features
scaler = preprocessing.StandardScaler()
train_data[['age', 'fare']] = scaler.fit_transform(train_data[['age','fare']])
test_data[['age', 'fare']] = scaler.transform(test_data[['age', 'fare']])

### Build Neural Network Classifier

In [ ]:
# Define features and target and convert to tensors
features = ['sex', 'age', 'fare']
target = 'survived'
train_features = tf.cast(train_data[features], tf.float32)
train_target = tf.cast(train_data[target], tf.int32)
test_features = tf.cast(test_data[features], tf.float32)
test_target = tf.cast(test_data[target], tf.int32)

# Build the binary classification model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(len(features),)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

### Model training

In [9]:
# Define optimizer and loss then train and evaluate
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_features, train_target, epochs=20, validation_split=0.2)
test_loss, test_acc = model.evaluate(test_features, test_target)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3395 - loss: 0.7373 - val_accuracy: 0.4238 - val_loss: 0.7065
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4430 - loss: 0.6997 - val_accuracy: 0.6524 - val_loss: 0.6771
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6245 - loss: 0.6598 - val_accuracy: 0.6667 - val_loss: 0.6532
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6894 - loss: 0.6310 - val_accuracy: 0.6810 - val_loss: 0.6348
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6840 - loss: 0.6122 - val_accuracy: 0.6571 - val_loss: 0.6226
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6721 - loss: 0.5923 - val_accuracy: 0.6619 - val_loss: 0.6129
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7159 - loss: 0.5614 - val_accuracy: 0.6810 - val_loss: 0.6049
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6930 - loss: 0.5680 - val_accuracy: 0.7143 - val_loss

### Model prediction

In [10]:
predictions = model.predict(test_features)
binary_predictions = (predictions > 0.5).astype(int)
results_df = pd.DataFrame({
  'Actual': test_target,
  'Predicted': binary_predictions.flatten(),
  'Sex': test_data['sex'].values,
  'Age': test_data['age'].values
})

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [11]:
# Analyze survival based on Sex and Age
sex_analysis = results_df.groupby('Sex').mean()
print("Survival rate based on Sex : \n")
print(sex_analysis, "\n")

age_analysis = results_df.groupby('Age').mean()
print("Survival rate based on Age : \n")
print(age_analysis)

Survival rate based on Sex : 

       Actual  Predicted       Age
Sex                               
0.0  0.680851   1.000000 -0.134849
1.0  0.190476   0.029762 -0.012996 

Survival rate based on Age : 

            Actual  Predicted       Sex
Age                                    
-1.397355  0.21875      0.375  0.640625
-1.331825  1.00000      1.000  0.000000
-1.322468  0.00000      0.000  1.000000
-1.299063  1.00000      1.000  0.000000
-1.289700  1.00000      1.000  1.000000
...            ...        ...       ...
 1.916481  0.00000      0.000  1.000000
 1.972648  0.00000      0.000  1.000000
 2.028815  1.00000      0.000  1.000000
 2.141148  0.00000      0.000  1.000000
 2.253482  0.00000      0.500  1.000000

[66 rows x 3 columns]
